# Visualizing Neo4j graphs in yFiles Graphs for Jupyter

Before using the graph widget, install all necessary packages.

In [ ]:
%pip install yfiles_jupyter_graphs --quiet
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass
%pip install neo4j --quiet
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

## How to import a graph
- either import the graph directly when initilizing: ```GraphWidget(graph=your_graph)```
- or use the ```w.import_graph(your_graph)``` function, if you already initilized a Widget called ```w```

## Notes about Neo4j importer
- Graph properties are ignored.
- Nodes and edges are identified by index.
- Node and edge properties are extracted from corresponding property maps.
- Default values for unset properties are used, due to the way graph tool properties work.
- Any additional data is stored in ```properties``` under the same name

In [ ]:
uri      = "neo4j+s://demo.neo4jlabs.com" 
user     = "movies"          # your user name 
                              # default is always "neo4j" 
                              # unless you have changed it. 
password = "movies"

driver = GraphDatabase.driver(uri=uri,auth=(user,password),database='movies')
session = driver.session(database='movies')


result = session.run("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20")

w = GraphWidget(graph=result.graph())
display(w)

In [ ]:
w.nodes

### Using imported data

As we can't really derive any information from this graph, we need to use the data available, that is currently stored in 'properties' to change the node and edge label mappings:

In [ ]:
w2 = GraphWidget(graph = result.graph())

def custom_node_label_mapping(index, node):
    """let the label be the name or the title"""
    properties = node.get('properties', {})
    return properties.get('title', properties.get('name', 'no label'));
w2.node_label_mapping = custom_node_label_mapping

def custom_relationship_label_mapping(index, node):
    """let the label be the role"""
    properties = node.get('properties', {})
    return properties.get('roles', ['no role'])[0];
w2.edge_label_mapping = custom_relationship_label_mapping

w2.hierarchic_layout()
display(w2)

An alternative method for changing the names and titles to labels, utilizing the "title" and "name" properties stored in the node properties:

In [ ]:
w3 = GraphWidget(graph = result.graph())

w3.node_label_mapping = lambda node: node['properties']['name'] if 'name' in node['properties'] else node['properties']['title']

w3.hierarchic_layout()
display(w3)